In this script i will generate from the vcf files from slim a table of allele counts per snp for all the populations as an input for baypass 
so the ouput should be, each row a snp positions and each colum the number of population X 2 since the first column is allele ref, the second alt allele

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import math 
import os

import matplotlib.pyplot as plt

import allel
import subprocess

In [3]:
def chek_indivn(df):
    # get the number of columns in the dataframe
    n_cols = len(df.columns)
    # create a list of column indices, each representing a pair of columns to be summed
    col_indices = [(2*i, 2*i+1) for i in range(n_cols//2)]   
    # sum the pairs of columns and concatenate the resulting series
    return pd.concat([df.iloc[:,i].add(df.iloc[:,j]) for (i,j) in col_indices], axis=1)

In [4]:
dir_cluster =  '/Users/tbellagio/tbellagio_cluster/simulations/slim_grenenet/vcf_slim/'

In [23]:
for optima in os.listdir(dir_cluster):
  
 optima_subp = os.listdir(dir_cluster + optima + '/') 
 if len(optima_subp)>0:
    print(optima)
    print(optima_subp)

optima0
['subp9.vcf', 'subp0.vcf', 'subp7.vcf', 'subp6.vcf', 'subp1.vcf', 'subp8.vcf', 'subp11.vcf', 'subp2.vcf', 'subp5.vcf', 'subp10.vcf', 'subp4.vcf', 'subp3.vcf']
optima3
['subp8.vcf', 'subp1.vcf', 'subp6.vcf', 'subp7.vcf', 'subp0.vcf', 'subp9.vcf', 'subp3.vcf', 'subp4.vcf', 'subp11.vcf', 'subp5.vcf', 'subp2.vcf', 'subp10.vcf']
optima1
['subp10.vcf', 'subp4.vcf', 'subp3.vcf', 'subp2.vcf', 'subp11.vcf', 'subp5.vcf', 'subp6.vcf', 'subp1.vcf', 'subp8.vcf', 'subp9.vcf', 'subp0.vcf', 'subp7.vcf']
optima2
['subp5.vcf', 'subp11.vcf', 'subp2.vcf', 'subp3.vcf', 'subp4.vcf', 'subp10.vcf', 'subp9.vcf', 'subp7.vcf', 'subp0.vcf', 'subp1.vcf', 'subp6.vcf', 'subp8.vcf']


In [9]:
### create a dictionary where 1 dataframe containing all the replicates for each optima is saved 

allele_counts = pd.DataFrame()
indiv_number = pd.DataFrame()
#  take all the vcf files in the folder 
for optima in ['optima0', ]: #os.listdir(dir_cluster):
    optima_subp = os.listdir(dir_cluster + optima + '/')
    subps = [i for i in optima_subp if '.vcf' in i ]
    
    for subp in subps:
        vcf = allel.read_vcf(dir_cluster + optima + '/' + subp)
        print(len(vcf['variants/POS']))
        genotypes = allel.GenotypeArray(vcf['calldata/GT'])
        ref_allele_counts = genotypes.count_alleles()[:, 0]
        alt_allele_counts = genotypes.count_alleles()[:, 1]

        subp0 = pd.DataFrame(data ={'ref_' + optima+subp[:-4]:ref_allele_counts, 'alt_'+ optima + subp[:-4]: alt_allele_counts}, index = vcf['variants/POS'], )
        number_of_ind = (subp0.iloc[0, 0] + subp0.iloc[0, 1]) / 2 
        print(number_of_ind)
        indiv_number[optima + subp[:-4]] = [number_of_ind]
        allele_counts = pd.concat([allele_counts, subp0], axis=1)

67354
900.0
55306
900.0
67331
900.0
67355
900.0
74063
900.0
67347
900.0
67353
900.0
67354
900.0
67332
900.0
73047
900.0
55306
900.0
55306
900.0


In [20]:
### if the alt is nan is because it is 0, and if the ref is nan is because it is the max number (number of individuals)

for col in allele_counts.columns:
    if 'ref_' in col: 
        allele_counts.loc[:,col] = allele_counts.loc[:, col].fillna(indiv_number.loc[0, col[4:]])
    elif 'alt_' in col:
        allele_counts.loc[:,col] = allele_counts.loc[:, col].fillna(0)

In [29]:
allele_counts = allele_counts.reset_index().rename(columns={'index': 'pos'})

In [31]:
allele_counts.to_csv('results_slim/pi0.001beta5/allele_counts.csv')

In [33]:
indiv_number.to_csv('results_slim/pi0.001beta5/pool_sizes.csv', index=None)